In [ ]:
## ResNet50, feature vector 뽑은 후 거리 계산  (lookbook -cloth_table 내에서..)

from PIL import Image
import os
import six.moves.cPickle as Pickle

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
import time

## 이미지 불러오기
with open('cloth_table.pkl', 'rb') as cloth_table:
    cloth_data = Pickle.load(cloth_table)
cloth_data
print(len(cloth_data)) # 8726


start = time.time()


## ResNet
features = []
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

for i in range(len(cloth_data)):#len(cloth_data)):
    img_path = '../data/' + cloth_data[i]   

    img = image.load_img(img_path, target_size=(224, 224)) 
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    feature = model.predict(x)
    # print(feature.shape)  # (1,2048)
    features.append(feature)
    
    if i % 50 == 0:
        print(i)

features_2048 = np.squeeze(np.array(features))


## Euclidean distance
dist = euclidean_distances(features_2048, features_2048)


## 계산시간
end = time.time() - start
print(end)  # 19990.61946296692


## 결과 저장
with open('resnet50_feature_2048.pkl', 'wb') as resnet50_feature_2048:
    Pickle.dump(features_2048, resnet50_feature_2048)
    
with open('resnet50_feature_2048_dist.pkl', 'wb') as resnet50_feature_2048_dist:
    Pickle.dump(dist, resnet50_feature_2048_dist)

In [ ]:
## 특정 이미지와 유사한 상위 5개 이미지 추출  (lookbook-cloth_table 내에서..)

from PIL import Image
import os
import six.moves.cPickle as Pickle
import numpy as np

## 이미지 불러오기
with open('cloth_table.pkl', 'rb') as cloth_table:
    cloth_data = Pickle.load(cloth_table)

## 결과 불러오기(feature vector)
with open('resnet50_feature_2048.pkl', 'rb') as resnet50_feature_2048:
    resnet50_feature_2048 = Pickle.load(resnet50_feature_2048)
# resnet50_feature_2048.shape  # (8726, 2048)
resnet50_feature_2048

## 결과 불러오기(euclidean distance)
with open('resnet50_feature_2048_dist.pkl', 'rb') as resnet50_feature_2048_dist:
    resnet50_feature_2048_dist = Pickle.load(resnet50_feature_2048_dist)
# resnet50_feature_2048_dist.shape  # (8726, 8726)
resnet50_feature_2048_dist


## 특정 이미지 선택 후 유사 이미지 보기(top5)
i = 4   # i번째 이미지 선택
top = 5

top_ind = np.argsort(resnet50_feature_2048_dist[i,])[:(top+1)]

j = 0
for ind in top_ind:     
    img_path = '../data/' + cloth_data[ind]   
    globals()['top{}'.format(j)] = Image.open(img_path)
    j += 1

top0  # 선택 이미지(=i번째 이미지)
# top1  # 선택 이미지와 가장 유사한 이미지 top 1
# top2  # 선택 이미지와 가장 유사한 이미지 top 2
# top3  
# top4
# top5

In [ ]:
## 새로운 이미지 추가시 유사 이미지 찾기

# find_images_ResNet(img_name, top_num)
# img_name: 새로운 이미지 파일명
# top_num: 0=자기자신, 1=가장 유사한 이미지 top1, 2=가장유사한 이미지 top2.

def find_images_ResNet(img_name, top_num):
    from PIL import Image
    import os
    import six.moves.cPickle as Pickle

    from keras.applications.resnet50 import ResNet50
    from keras.preprocessing import image
    from keras.applications.resnet50 import preprocess_input, decode_predictions
    
    import numpy as np
    from sklearn.metrics.pairwise import euclidean_distances

    ## ResNet
    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

    img_path = '../ex/' + img_name + '.png'  # 새로운 이미지 저장된 경로 지정

    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    ex_feature = model.predict(x)


    ## 기존 이미지(lookbook -cloth_table)의 feature vector 불러오기   
    with open('resnet50_feature_2048.pkl', 'rb') as resnet50_feature_2048:
        resnet50_feature_2048 = Pickle.load(resnet50_feature_2048)


    ## 이미지간 거리 계산(euclidean distance)
    dist = euclidean_distances(ex_feature, resnet50_feature_2048)
    dist = np.squeeze(dist)


    ## 기존 이미지 불러오기
    with open('cloth_table.pkl', 'rb') as cloth_table:
        cloth_data = Pickle.load(cloth_table)

        
    ## 유사 이미지 선택   
    top_ind = np.argsort(dist)

    if (top_num == 0):
        result = Image.open(img_path)  
    if (top_num > 0):
        result = Image.open('../data/' + cloth_data[top_ind[top_num-1]]) 
    return result


## 함수 실행
# find_images_ResNet("ex2",0)  # 내가 넣은 새로운 이미지 출력
# find_images_ResNet("ex2",1)  # 새로운 이미지와 가장 유사한 이미지 top1
# find_images_ResNet("ex2",2)  # 새로운 이미지와 가장 유사한 이미지 top2 
# find_images_ResNet("ex2",3)  
# find_images_ResNet("ex2",4)  
find_images_ResNet("ex2",5)  